# Elevation Covariate

Read elevation data by a bounding box tile for stacking with other covariates in analysis.
Initial version uses SRTM v3 available as Cloud Optimized Geotiffs in MAAP.

In [13]:
from maap.maap import MAAP
maap = MAAP()
from pprint import pprint

In [4]:
# generated from this EDSC search: https://ade.maap-project.org:30052/search/granules?p=C1200271393-NASA_MAAP&q=SRTM&tl=1598404169!4!!
collection_info = maap.searchCollection(concept_id="C1200271393-NASA_MAAP", limit=1000)

In [16]:
short_name = collection_info[0]['Collection']['ShortName']


In [19]:
bbox = '-101,45,-100,46'
granule_results = maap.searchGranule(short_name=short_name, bounding_box=bbox , limit=20)

In [ ]:
len(granule_results)

## Inspect the results


In [30]:
import geopandas as gpd
import folium
import shapely as shp


/opt/conda/lib/python3.7/site-packages/geopandas/_compat.py:88: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.8.1-CAPI-1.13.3). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


In [44]:
def make_polygons(item):
    bounds = item['Granule']['Spatial']['HorizontalSpatialDomain']['Geometry']['BoundingRectangle']
    item_bbox = [float(value) for value in bounds.values()]
    bbox_polygon = shp.geometry.box(
        item_bbox[0], 
        item_bbox[1],
        item_bbox[2], 
        item_bbox[3]
    )
    return bbox_polygon

In [46]:
item = granule_results[0]
bounds = item['Granule']['Spatial']['HorizontalSpatialDomain']['Geometry']['BoundingRectangle']
check = make_polygons(item)
geometries = gpd.GeoSeries([make_polygons(item) for item in granule_results])
geometries

0    POLYGON ((-100.99972 45.00028, -100.99972 43.9...
1    POLYGON ((-98.99972 45.00028, -98.99972 43.999...
2    POLYGON ((-98.99972 47.00028, -98.99972 45.999...
3    POLYGON ((-100.99972 46.00028, -100.99972 44.9...
4    POLYGON ((-100.99972 47.00028, -100.99972 45.9...
5    POLYGON ((-99.99972 45.00028, -99.99972 43.999...
6    POLYGON ((-99.99972 46.00028, -99.99972 44.999...
7    POLYGON ((-98.99972 46.00028, -98.99972 44.999...
8    POLYGON ((-99.99972 47.00028, -99.99972 45.999...
dtype: geometry

## Read the data for the Box

For each result read the subwindow. After reading all the subwindows, merge them together as a single  array. 

Q: is it easier to make a vrt of the files and then request the subwindow, or does that waste time?